### Makeathon Model Building

This notebook is for building the model for our makeathon prompt. This notebook will follow vaguely the tutorial found at https://medium.com/swlh/emotion-detection-using-opencv-and-keras-771260bbd7f7

In [1]:
#Import the necessary libraries
import pandas as pd
import numpy as np
import keras
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
from keras.optimizers import RMSprop,SGD,Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
from keras.regularizers import l2
import os
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:
#Now for some definitions we need for later
num_classes=7
img_rows,img_cols=48,48
batch_size= 64

In [4]:
#Create image generators for image augmentation and rescaling for both train and validation datasets
train_datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # dimesion reduction
        rotation_range=30,  # randomly rotate images in the range 20 degrees
        zoom_range = 0.2, # Randomly zoom image 20%
        width_shift_range=0.1,  # randomly shift images horizontally 10%
        height_shift_range=0.1,  # randomly shift images vertically 10%
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
#(
#    rotation_range=40,
#    shear_range=0.3,
#    zoom_range=0.3,
#    width_shift_range=0.4,
#    height_shift_range=0.4,
#    horizontal_flip=True,
#    fill_mode='nearest')

In [5]:
#Read in and prep all the data
def prepare_data(data): 
    image_array = np.zeros(shape=(len(data), 48, 48))
    image_label = np.array(list(map(int, data['emotion'])))
    
    for i, row in enumerate(data.index):
        image = np.fromstring(data.loc[row, 'pixels'], dtype=int, sep=' ')
        image = np.reshape(image, (48, 48))
        image_array[i] = image
        
    return image_array, image_label

data = pd.read_csv('fer2013.csv')

train_image_array, train_image_label = prepare_data(data[data['Usage']=='Training'])
val_image_array, val_image_label = prepare_data(data[data['Usage']=='PrivateTest'])
test_image_array, test_image_label = prepare_data(data[data['Usage']=='PublicTest'])

In [6]:
train_images = train_image_array.reshape((train_image_array.shape[0], 48, 48, 1))
X_train = train_images.astype('float32')/255
val_images = val_image_array.reshape((val_image_array.shape[0], 48, 48, 1))
X_val = val_images.astype('float32')/255
test_images = test_image_array.reshape((test_image_array.shape[0], 48, 48, 1))
X_test = test_images.astype('float32')/255

y_train = to_categorical(train_image_label)
y_val = to_categorical(val_image_label)
y_test = to_categorical(test_image_label)

In [7]:
#Fit our image data generators
train_datagen.fit(X_train)

In [8]:
#We're going to build a basic CNN using Keras Sequential models
reg = l2(.001)
model = Sequential()
#Block-1 
model.add(Conv2D(32,(3,3),padding='same', kernel_initializer='he_normal',
                 kernel_regularizer = reg, input_shape=(img_rows,img_cols,1), activation = 'elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same', kernel_initializer='he_normal',
                 kernel_regularizer = reg, input_shape=(img_rows,img_cols,1), activation='elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
#Block-2
model.add(Conv2D(64,(3,3),padding='same', kernel_initializer='he_normal',
                 kernel_regularizer = reg, activation='elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same', kernel_initializer='he_normal',
                 kernel_regularizer = reg, activation = 'elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
#Block-3
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal',
                 kernel_regularizer = reg, activation='elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal',
                 kernel_regularizer = reg, activation='elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
#Block-4
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal', 
                 activation='elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal', 
                 activation='elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
#Block-5
model.add(Flatten())
model.add(Dense(256,kernel_initializer='he_normal', activation='elu'))
model.add(Dense(128,kernel_initializer='he_normal', activation='elu'))
model.add(Dropout(0.5))
#Block-6
#model.add(Dense(32,kernel_initializer='he_normal', activation='elu'))
#model.add(Dense(16,kernel_initializer='he_normal', activation='elu'))
#model.add(Dropout(0.5))
#Block-7
model.add(Dense(num_classes,kernel_initializer='he_normal', activation='softmax'))
#Show a summary of the model
model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 24, 24, 32)        0         
______________________________________

In [9]:
#Create some callbacks for use during training
#This came straight from the article, I didn't change anything here
checkpoint = ModelCheckpoint('EmotionDetectionModel_v20.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)
earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=4,
                          verbose=1,
                          restore_best_weights=True
                          )
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)
callbacks = [checkpoint,earlystop,reduce_lr]

In [ ]:
#Now, we'll compile and fit the model
opt = Adam()
model.compile(optimizer = opt, loss='categorical_crossentropy', metrics=['accuracy'])

#Now fit the model with fit
nb_train_samples = 28709
nb_validation_samples = 3589
epochs= 93
history=model.fit(
 X_train, y_train,
 steps_per_epoch=nb_train_samples//batch_size,
 epochs=epochs,
 callbacks=callbacks,
 validation_data=(X_val, y_val),
 validation_steps = nb_validation_samples//batch_size)
model.save('makeathon_model_v20(l2reg-Dropout25-BS64-256x2-biggen).h5')


Train on 28709 samples, validate on 3589 samples
Epoch 1/93


In [ ]:
#Plot the training and validation loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper right')
plt.savefig('loss_v20.png',bbox_inches= 'tight')
plt.show()

In [ ]:
#Plot the training and validation accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='lower right')
plt.savefig('accuracy_v20.png',bbox_inches= 'tight')
plt.show()